# Republic frase modellen

Doel:

- informatie extractie uit de resoluties, gebruik makend van de repetitieve aspecten van de teksten, formulaische uitdrukkingen, signaalwoorden en onderwerpstermen.
- het opbouwen van collectie-specifieke frase-modellen (lijstjes met termen en frasen, varianten daarvan en metadata categorieën en labels) zodat we informatie kunnen extraheren en om kunnen vormen tot inhoudelijke metadata voor ontsluiting van de individuele resoluties. 

Aanpak: harvesting phrases o.b.v. frequentielijsten

Overlap/verschil tussen jaren:

- tussen twee opvolgende jaren
- tussen twee jaren met een decennium ertussen
- tussen twee jaren met X jaren ertussen

Overlap tussen resoluties en respectenlijsten:

- in hoeverre komen termen uit de respecten letterlijk voor in de resoluties?
- vergelijk e.g. automatische extractie en respecten. Dit vergt ook interactie met Team Text en hun NER activiteiten

## Overlap analyseren

**doel:** bepalen welke termen structureel vaak voorkomen en hoe ze kunnen worden gebruikt om de inhoud van resoluties te analyseren

Lijstjes:

- hoedanigheden, rollen (ambassadeur, graaf, suppliant)
    - dit kan gebruikt worden om o.a. persoonsnamen te spotten (dus nuttig voor NER trainen of resultaten opschonen)
    - signaalwoorden/-frasen:
        - <ambassadeur> in, uit, van, bij <land/hof>
        - uit, naar, tot, te <plaats>
        - gecommiteerd/commissie
- documenten:
    - Placaat
    - Rapport
    - Paspoort
- onderwerpen:
    - financiën
    - rechten


In [2]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

In [5]:
# Laad de benodigde modules in
import re
import json
from collections import Counter
from collections import defaultdict
from typing import Union
from republic.model.inventory_mapping import get_inventory_by_num
from republic.config.republic_config import base_config, set_config_inventory_num
import republic.elastic.republic_elasticsearch as rep_es

# Een handige 'Key Word In Context' functie om keywords/frasen en omringende tekst te zien
def get_keyword_context(text: str, keyword: str, 
                        context_size: int = 3, 
                        prefix_size: Union[None, int] = None, 
                        suffix_size: Union[None, int] = None):
    if not prefix_size:
        prefix_size = context_size
    if not suffix_size:
        suffix_size = context_size
    text = re.sub(r'\s+',' ', text)
    prefix_pattern = r'(\w*\W*){' + f'{prefix_size}' + '}$'
    suffix_pattern = r'^(\w*\W*){' + f'{suffix_size}' + '}'
    contexts = []
    for match in re.finditer(r'\b' + keyword + r'\b', text):
        prefix_window = text[:match.start()]
        suffix_window = text[match.end():]
        prefix_terms = prefix_window.strip().split(' ')
        suffix_terms = suffix_window.strip().split(' ')
        prefix = ' '.join(prefix_terms[-prefix_size:])# re.search(prefix_pattern, prefix_window)
        suffix = ' '.join(suffix_terms[:suffix_size])# re.search(suffix_pattern, suffix_window)
        context = {
            'keyword': keyword,
            'prefix': prefix,
            'suffix': suffix,
            'keyword_offset': match.start(),
            'prefix_offset': match.start() - len(prefix),
            'suffix_offset': match.end()
        }
        contexts.append(context)
    return contexts


es_republic = rep_es.initialize_es(host_type='external', timeout=60)



In [6]:
# kies een inventaris nummer om te doorzoeken
inv_num = 3825
# de base_dir is niet relevant maar wel nodig voor het maken van een config object
base_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/"
# genereer een config object voor het specifieke inventaris nummer dat je wilt doorzoeken
inv_config = set_config_inventory_num(base_config, inv_num, base_dir, ocr_type='pagexml')

# haal alle paragraphs van het inventaris op uit de index
#docs = rep_es.retrieve_paragraphs_by_inventory(es_republic, inv_num, inv_config)
paragraph_index = 'pagexml_meeting_paragraphs'
query = {'query': {'match': {'metadata.inventory_num': inv_num}}, 'size': 10000}
#response = es_republic.search(index=paragraph_index, body=query)
docs = [hit['_source'] for hit in response['hits']['hits']]

print('number of paragraphs:', len(docs))

if len(docs) > 0:
    # hoe ziet een paragraph document eruit?
    print(json.dumps(docs[0], indent=4))

number of paragraphs: 4405
{
    "metadata": {
        "textregion_id": "NL-HaNA_3825_0088-page-175-col-0-tr-0",
        "inventory_num": 3825,
        "scan_num": 88,
        "scan_id": "NL-HaNA_3825_0088",
        "page_num": 175,
        "page_id": "NL-HaNA_3825_0088-page-175",
        "column_index": 0,
        "column_id": "NL-HaNA_3825_0088-page-175-col-0",
        "meeting_date": "1770-01-01",
        "meeting_id": "meeting-1770-01-01-session-1",
        "id": "NL-HaNA_3825_0088-page-175-col-0-tr-0",
        "scan_type": "pagexml",
        "page_type": "resolution_page"
    },
    "coords": {
        "left": 2423,
        "right": 3306,
        "top": 3004,
        "bottom": 3167,
        "height": 163,
        "width": 883
    },
    "lines": [
        "NL-HaNA_3825_0088-page-175-col-0-tr-0-line-0",
        "NL-HaNA_3825_0088-page-175-col-0-tr-0-line-1",
        "NL-HaNA_3825_0088-page-175-col-0-tr-0-line-2"
    ],
    "text": "Tumeert en. te\u00fbrrefteert  zvn>:de;dDepsches d

## Simpele frequentielijsten

Een eerste stap is kijken wat de frequente woorden zijn in de resoluties van een specifiek inventaris. 

In de resoluties beginnen veel domeinspecifieke inhoudstermen met een hoofdletter. Dat is een goed eerste filter om inzicht te krijgen in de inhoud.

In [7]:
# maak een nieuwe counter aan
term_freq = Counter()

# itereer over all documenten (= paragraphs)
for doc in docs:
    # splits de tekst op alles wat geen alpha-numerical character is
    terms = re.split(r'\W+', doc['text'])
    # een 'list comprehension' om termen te filteren die met een hoofdletter te beginnen
    terms = [term for term in terms if term != '' and term[0].isupper()]
    # update de counter met de lijst van termen in dit document
    term_freq.update(terms)

# print een overzicht van de 100 meest frequente termen
for term, freq in term_freq.most_common(100):
    # print term (links uitgelijnd met een breedte van 30 karakters) en frequentie (rechts uitgelijnd met 6 karakters)
    print(f'{term: <30}{freq: >6}')

Hoog                            5854
Mog                             5440
Heeren                          2277
WAAR                            1925
Heere                           1476
Dat                             1307
Provincie                       1197
Gedeputeerden                   1049
Suppliant                       1004
Vergaderinge                     961
Ntfangen                         960
Raad                             933
Hof                              887
Missive                          804
Stad                             798
En                               748
Supplianten                      692
Refolutie                        678
Staaten                          672
Resolutie                        633
Mos                              623
Copie                            586
Staate                           579
Admiraliteit                     572
Requeste                         548
Mogende                          546
Lynden                           526
Z

Termen hoeven geen individuele woorden te zijn, maar kunnen ook woord bi-grammen of groter woord n-grammen zijn.

In [13]:
# maak een nieuwe counter aan
term_freq = Counter()

ngram_size = 3

# itereer over all documenten (= paragraphs)
for doc in docs:
    # splits de tekst op alles wat geen alpha-numerical character is
    terms = re.split(r'\s+', doc['text'])
    # creeër ngrammen door elk woor te verbinden met de n-1 opvolgende woorden
    ngram_terms = [' '.join(terms[ti:ti+ngram_size]) for ti, term in enumerate(terms[:-(ngram_size-1)])]
    # een 'list comprehension' om termen te filteren die met een hoofdletter te beginnen
    terms = [term for term in ngram_terms if term != '' and term[0].isupper()]
    # update de counter met de lijst van termen in dit document
    term_freq.update(terms)

# print een overzicht van de 100 meest frequente termen
for term, freq in term_freq.most_common(100):
    # print term (links uitgelijnd met een breedte van 30 karakters) en frequentie (rechts uitgelijnd met 6 karakters)
    print(f'{term: <30}{freq: >6}')

WAAR op gedelibereert            805
WAAR op geen                     642
Ntfangen een Missive             364
Copie van de                     357
Missive van den                  349
Gedeputeerden van de             303
Gedeputeerden tot de             300
Heeren Gedeputeerden van         294
Heeren Staaten van               279
Raad van Staate                  251
Resolutie van den                235
Hoog Mog. te                     227
Miffive van den                  226
Hoog Mog. Gedeputeerden          223
Mog. Gedeputeerden tot           218
Collegie ter Admiraliteit        205
Raad van Staate,                 203
Refolutie van den                202
Vergaderinge rapport te          190
Depeches daar uit                184
Hof van fijne                    181
Majesteit den Koning             166
Envoyé aan het                   162
Miflive van den                  157
Heeren van Lynden                156
Majefteit den Koning             155
Ntfangen een Miffive             153
G

## Zoeken met Keywords

Als je interessante woorden/ngrammen hebt gevonden, kun je ze in context zoeken om te kijken wat frequente frasen zijn.

In [16]:
# maak counters voor de tekst voorafgaand en volgend op een keyword
prefix_freq = Counter()
suffix_freq = Counter()

# zomaar een lijstje veel voorkomende woorden/frasen
term = 'Schip'
term = 'Collegie'
term = 'Gecommitteerden'
term = 'Regiment'
term = 'Regenten'
term = 'Pafport'
term = 'Placaat'
term = 'ophef'
term = 'Raad'
term = 'Hoogh Mogende extraordinaris'
term = 'Collegie ter Admiraliteyt'
term = 'Heeren Gedeputeerden van'
term = 'Generaliteyts Reeckenkamer'
term = 'Compagnie'
term = 'Magistraet' # van
term = 'Ambaffadeur' # (van|aan het hof van|tot)
term = 'Hoog Mog. Gedeputeerden'

# itereer over de documenten
for doc in docs:
    # zoek de term en bijbehorende context op in het document
    for context in get_keyword_context(doc['text'], term, context_size=5):
        # tel hoe vaak elke prefix/suffix voorkomt
        prefix_freq.update([context['prefix']])
        suffix_freq.update([context['suffix']])
    
# toont een overzicht van de meest frequente prefixes en suffixes
for prefix, freq in prefix_freq.most_common(10):
    print(f'{prefix: <30}{freq: >6}')

print('\n')

for suffix, freq in suffix_freq.most_common(100):
    print(f'{suffix: <30}{freq: >6}')



tot Hemmen, en andere haar        18
Heeren Nagel, en andere haar      13
Heeren Brantsen, en andere haar    12
tot Kell, en andere haar           7
van Randwyck, en andere haar       7
de Pagniet, en andere haar         6
Heeren Brantfen, en andere haar     5
van Randwyk, en andere haar        5
Heeren Raad, en andere haar        5
Hemmen , en andere haar            4


tot de saaken van de              71
tot de faaken van de              40
tot de saaken van Vlaanderen,      6
tot de faaken van het              5
tot de saaken van het              4
tot de saken van de                2
tot de shaken van de               2
tot de saaken                      2
tot de buitenlandfche faaken, om     2
                                   2
tot de faa: ken van                2
tot de buitenlandsche saaken, om     2
tot de buiten. landsche saaken,     2
tot de faaken van. de              2
tot de faaken-van'de Zee, om       1
, geweest zynde na Vlaanderen,     1
tot de faaken van Vlaanderen’

## Fuzzy zoeken

Omdat er allerlei OCR fouten en spellingsvarianten voorkomen van een keyword/frase, mis je met exact zoeken allerlei voorkomens. De Fuzzy search module biedt mogelijkheden om inexact te zoeken naar tekst strings die lijken op een geven set van keywords/frasen.

In de configuratie kun je aangeven hoeveel een string van een keyword mag afwijken om als match te tellen.

In [34]:
# importeer de fuzzy keyword searcher class en de phrase model class
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyKeywordSearcher
from republic.fuzzy.fuzzy_phrase_model import PhraseModel

# configuratie van de fuzzy searcher
fuzzy_search_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# creeër een nieuwe fuzzy searcher met bovenstaande configuratie 
fuzzy_searcher = FuzzyKeywordSearcher(fuzzy_search_config)


# keywords mag een lijst van strings zijn:
keywords = [
    'Raad van Staate',
    'Ambaffadeur aan het hof van',
    'Generaliteyts Reeckenkamer'
]

# of je kunt per keyword extra gegevens toevoegen:
# 'variants': variante voorkomens die je onder hetzelfde keyword wilt tellen
# 'label': een categorizerend label
# 'distractors': keywords die qua tekst-afstand op het keyword lijken maar toch echt iets anders zijn
keywords = [
    {
        'keyword': 'Raad van Staate',
        'variants': [
            'Raedt van State'
        ],
        'label': 'raad'
    },
    {
        'keyword': 'Hoogh Mogende Gedeputeerden tot de Militaire saecken',
        'label': 'gedeputeerden'
    },
    {
        'keyword': 'Hoogh Mogende Gedeputeerden tot de Buytenlandtsche saecken',
        'label': 'gedeputeerden',
        'variants': [
            'Hoog Mog. Gedeputeerden tot de buitenlandsche saaken'
        ]
    },
    {
        'keyword': 'Ambassadeur aan het hof van',
        'label': 'ambassadeur',
        'variants': [
            'Ambassadeur van hare'
        ]
    },
    {
        'keyword': 'Generaliteyts Reeckenkamer',
        'label': 'organisatie'
    }
]

# transformeer je lijst met keywords in een phrase model
phrase_model = PhraseModel(model=keywords)
# indexeer de keywords uit het phrase model met de fuzzy searcher
fuzzy_searcher.index_keywords(phrase_model.get_keywords())
# indexeer eventuele varianten van de keywords
fuzzy_searcher.index_spelling_variants(phrase_model.variants)

In [35]:
import json

# itereer over de documenten
for doc in docs:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates(doc['text'], use_word_boundaries=True, include_variants=True)
    # check of er matches zijn en print de tekst en de match objecten
    if len(matches) > 0:
        print(doc['text'])
        for match in matches:
            print(json.dumps(match, indent=4))
        break


derfeeft van St. ’Cätharina, ‘en de inaugura tie van dênieu we Ridderorder van St. George, haar Hoog Mog. permiffie om de koften daat van ‘in declaratie te brengen. WAAR op gedelibereert zynde, is goed gevonden en verftaan ‚ dat aan gemelde Îleere Grave van Rechteren fal worden gerefcribeert, dat haar Hoog Mog. 2an-hem permitteeren om de kolten van’ gemelde [Ilaminatien te mogen brengen ín fijne. De: catatúie, alwaar defelve aan hem fullen worTs 9755 Dogs den gevalideert. En fal Extra van deefe haar Hoog Mog. Refolutie gefonden worden aan den Räad van Staate en de Generaliteit ReeKenkamer, om te {trekken tot der {elver narigtinge. eb: Rn Ntfansen een Miffive van den Heere CN Bijchop en Purft te Paderborn; ge fchreeven te Neuhaus den 20 der voörleeden maand, houdéhde een Nieuwe: jaarswenfch. WAAR ‘op gedelibereert zynde, is goedgevonden en verftaan, dat de vooffë’ Mis. five met een Röfcriptie in civilé ‘termen {àl boshotr sb ge worden beantwoord. En Heer Bergsma; ‘ter Vergaderinge EJ or

In [41]:
# creeër counters voor de varianten van elk keyword
variants = defaultdict(Counter)

# itereer over de documenten
for doc in docs[:200]:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates(doc['text'], use_word_boundaries=True, include_variants=True)
    # voor elke match, tel de matchende string als variant van het matchende keyword
    for match in matches:
        variants[match['match_keyword']].update([match['match_string']])

# toon voor elk van de gevonden keywords een overzicht van de meest frequente varianten
for keyword in variants:
    print('Keyword:', keyword)
    print('------------------------------------')
    for variant, freq in variants[keyword].most_common():
        print(f'{variant: <20}\t{freq: >6}')
    print('\n')


Keyword: Raad van Staate
------------------------------------
Raad van Staate     	    38
Raad wan Staate     	     2
Räad van Staate     	     1
Raäd va Staate      	     1
Raad -van Staate    	     1
Raad van Staaté     	     1
Faad van’ Staate    	     1
aad v an Staate     	     1
Raad van Stäate     	     1
Rad van’ Staate     	     1
Rad van Staate      	     1
Raad van Staat      	     1
Raad van State      	     1


Keyword: Generaliteyts Reeckenkamer
------------------------------------
Generaliteits Reekenkamer	    15
Generaliteits Reekenka. mer	     1
Generaliteits Réekenkamer	     1
Generaliteits. Reekenkaamer	     1
Generaliceits Reekeikamer	     1
Generaliceits Reekenkamer	     1
Generaliteits Ree kenkamer	     1
Generalitcits Reekénkaamer	     1


Keyword: Ambassadeur aan het hof van
------------------------------------
Ambassadeur aan het Hof van	     2
Ambafladeur -aan‘het Hof van	     1
Ambafladeur aan het, Hof van	     1
Ambassadeur aan het Hos van	     1
AmbafTadeur

In [39]:
# controleer of je pandas geinstalleerd hebt
import pandas as pd


labels = {keyword['keyword']: keyword['label'] for keyword in keywords}

# houdt voor alle matches metadata bij
hits = []

# itereer over de documenten
for doc in docs[:200]:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates(doc['text'], use_word_boundaries=True, include_variants=True)
    # voor elke match, tel de matchende string als variant van het matchende keyword
    for match in matches:
        hit = {
            'meeting_date': doc['metadata']['meeting_date'], 
            'inventory_num': doc['metadata']['inventory_num'], 
            'page_num': doc['metadata']['page_num'],
            'keyword': match['match_keyword'],
            'variant': match['match_term'],
            'match_string': match['match_string'],
            'label': labels[match['match_keyword']]
        }
        hits.append(hit)
        
columns = list(hits[0].keys())
data = {column: [hit[column] for hit in hits] for column in columns}
df = pd.DataFrame(data)

In [40]:
df

,meeting_date,inventory_num,page_num,keyword,variant,match_string,label
0,1770-01-01,3825,177,Raad van Staate,Raad van Staate,Räad van Staate,raad
1,1770-01-01,3825,177,Raad van Staate,Raad van Staate,Raad van Staate,raad
2,1770-01-01,3825,177,Generaliteyts Reeckenkamer,Generaliteyts Reeckenkamer,Generaliteits Reekenkamer,organisatie
3,1770-01-01,3825,178,Ambassadeur aan het hof van,Ambassadeur aan het hof van,Ambafladeur -aan‘het Hof van,ambassadeur
4,1770-01-01,3825,179,Raad van Staate,Raad van Staate,Raad van Staate,raad
...,...,...,...,...,...,...,...
77,1770-01-19,3825,211,Raad van Staate,Raad van Staate,Raad van Staate,raad
78,1770-01-22,3825,212,Ambassadeur aan het hof van,Ambassadeur aan het hof van,Aanbassadeur aan het Hof van,ambassadeur
79,1770-01-22,3825,213,Raad van Staate,Raad van Staate,Raad van Staat,raad
80,1770-01-22,3825,213,Raad van Staate,Raad van Staate,Raad van State,raad
